<a href="https://colab.research.google.com/github/Adrianfc22/Final_Test_Vision/blob/main/inception_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import zipfile

In [3]:
data_dir = "/content/drive/MyDrive/Colab_Notebooks/archive/brain_tumor_dataset"
TEST = 'test'
TRAIN = 'train'
VAL ='val'

In [4]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import os
import copy

import torch
import torch.nn as nn
import torchvision

from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
import seaborn as sns

def transform_data(phase):
    if phase == TRAIN:
        transform = transforms.Compose([
            transforms.Resize(300),
            transforms.CenterCrop(300),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
        
    if phase == VAL:
        transform = transforms.Compose([
            transforms.Resize(300),
            transforms.CenterCrop(300),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    
    if phase == TEST:
        transform = transforms.Compose([
            transforms.Resize(300),
            transforms.CenterCrop(300),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])        
        
    return transform


device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
print(device)

cuda:0


In [5]:

data_transforms = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform_data(x)) 
                  for x in [TRAIN, VAL, TEST]}


dataloaders = {TRAIN: torch.utils.data.DataLoader(data_transforms[TRAIN], batch_size = 4, shuffle=True), 
               VAL: torch.utils.data.DataLoader(data_transforms[VAL], batch_size = 1, shuffle=True), 
               TEST: torch.utils.data.DataLoader(data_transforms[TEST], batch_size = 1, shuffle=True)}


dataset_sizes = {x: len(data_transforms[x]) for x in [TRAIN, VAL]}

print("Dataset sizes: ", dataset_sizes)
classes = data_transforms[TRAIN].classes
print("Dataset classes in TRAIN: ", classes)
classes = data_transforms[VAL].classes
print("Dataset classes in VAL: ", classes)

Dataset sizes:  {'train': 208, 'val': 18}
Dataset classes in TRAIN:  ['no', 'yes']
Dataset classes in VAL:  ['no', 'yes']


Model Inception

In [6]:
inception = models.inception_v3(pretrained=True)
inception.aux_logits=False
inception.fc = nn.Linear(2048, 2)
print(inception)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth



Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), str

In [8]:
import torch.optim as optim
inception = inception.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(inception.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01)

In [11]:
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    
    for epoch in range(num_epochs):
        print("Epoch: {}/{}".format(epoch+1, num_epochs))
        print("="*10)
        
        
        for phase in [TRAIN, VAL]:

            
            if phase == TRAIN:
                scheduler.step()
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            
            for data in dataloaders[phase]:
                images, labels = data
                
                
                images = images.to(device)
                labels = labels.to(device)

                
                optimizer.zero_grad()

                
                with torch.set_grad_enabled(phase==TRAIN):
                    
                    predicted = model(images)
                    _, preds = torch.max(predicted, 1)
                    loss = criterion(predicted, labels)

                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                
                running_loss += loss.item() * images.size(0)

                
                running_corrects += torch.sum(preds == labels.data)

            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

              
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_weights = copy.deepcopy(model.state_dict())

    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_weights)
    return model

n_epochs = 35

inception = train_model(inception, criterion, optimizer, exp_lr_scheduler, num_epochs=n_epochs)

Epoch: 1/35


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 0.6072 Acc: 0.6683
val Loss: 0.7446 Acc: 0.5556
Epoch: 2/35
train Loss: 0.4262 Acc: 0.8077
val Loss: 0.4212 Acc: 0.8889
Epoch: 3/35
train Loss: 0.2614 Acc: 0.9038
val Loss: 0.3734 Acc: 0.8889
Epoch: 4/35
train Loss: 0.1650 Acc: 0.9375
val Loss: 0.2542 Acc: 0.8889
Epoch: 5/35
train Loss: 0.2791 Acc: 0.9135
val Loss: 0.3769 Acc: 0.8333
Epoch: 6/35
train Loss: 0.1620 Acc: 0.9471
val Loss: 0.3830 Acc: 0.8333
Epoch: 7/35
train Loss: 0.1238 Acc: 0.9712
val Loss: 0.3705 Acc: 0.8333
Epoch: 8/35
train Loss: 0.0898 Acc: 0.9808
val Loss: 0.4141 Acc: 0.7778
Epoch: 9/35
train Loss: 0.0948 Acc: 0.9663
val Loss: 0.4855 Acc: 0.7778
Epoch: 10/35
train Loss: 0.1655 Acc: 0.9327
val Loss: 0.3735 Acc: 0.8333
Epoch: 11/35
train Loss: 0.1256 Acc: 0.9567
val Loss: 0.3613 Acc: 0.8889
Epoch: 12/35
train Loss: 0.1142 Acc: 0.9615
val Loss: 0.5343 Acc: 0.7778
Epoch: 13/35
train Loss: 0.0736 Acc: 0.9856
val Loss: 0.3998 Acc: 0.7778
Epoch: 14/35
train Loss: 0.1305 Acc: 0.9663
val Loss: 0.3343 Acc: 0.9444